In [1]:
import os
import ast
import sys
import json
import torch
import pickle
import subprocess

sys.path.append('../')

import pandas as pd

from pathlib import Path
from tqdm import tqdm
from llamafactory.chat import ChatModel
from llamafactory.extras.misc import torch_gc
from sklearn.metrics import classification_report
#from utils.post_processing import post_process


In [2]:
try:    
    assert torch.cuda.is_available() is True
    
except AssertionError:
    
    print("Please set up a GPU before using LLaMA Factory...")

In [ ]:
CURRENT_DIR = Path.cwd()
FT_DIR = CURRENT_DIR / "emotion_analysis_comics" / "finetuning"
DATASET_DIR = CURRENT_DIR / "emotion_analysis_comics" / "finetuning" / "datasets"

ERC_DIR = FT_DIR.parent
LLAMA_FACTORY_DIR = ERC_DIR / "LLaMA-Factory"

BASE_MODEL = "unsloth/Qwen2.5-32B-Instruct-bnb-4bit"
LOGGING_DIR = FT_DIR / "training_logs"
OUTPUT_DIR = FT_DIR / "saved_models" / f"""comics35_pg_nb_{BASE_MODEL.split("/")[1]}"""

In [4]:
DATASET_DIR = CURRENT_DIR / "emotion_analysis_comics" / "finetuning" / "datasets"

In [5]:
train_dataset_name = f"""comics35_utterance_pg_train.json"""
test_dataset_name = f"""comics35_utterance_pg_test.json"""

train_dataset_file = DATASET_DIR / train_dataset_name
test_dataset_file = DATASET_DIR / test_dataset_name

In [6]:
train_dataset_file

PosixPath('/Utilisateurs/umushtaq/emotion_analysis_comics/finetuning/datasets/comics35_utterance_pg_train.json')

In [7]:

if not os.path.exists(os.path.join(FT_DIR, "model_args")):
    os.mkdir(os.path.join(FT_DIR, "model_args"))

train_file = FT_DIR / "model_args" / f"""{train_dataset_name.split(".")[0].split("train")[0]}{BASE_MODEL.split("/")[1]}.json"""

In [8]:
dataset_info_line =  {
  "file_name": f"{train_dataset_file}",
  "columns": {
    "prompt": "instruction",
    "query": "input",
    "response": "output"
  }
}

In [9]:
with open(os.path.join(LLAMA_FACTORY_DIR, "data/dataset_info.json"), "r") as jsonFile:
    data = json.load(jsonFile)

data["comics"] = dataset_info_line

with open(os.path.join(LLAMA_FACTORY_DIR, "data/dataset_info.json"), "w") as jsonFile:
    json.dump(data, jsonFile)

In [10]:
NB_EPOCHS = 8

In [ ]:
args = dict(
    
  stage="sft",                           # do supervised fine-tuning
  do_train=True,

  model_name_or_path=BASE_MODEL,         # use bnb-4bit-quantized Llama-3-8B-Instruct model
  num_train_epochs=NB_EPOCHS,            # the epochs of training
  output_dir=str(OUTPUT_DIR),                 # the path to save LoRA adapters
  overwrite_output_dir=True,             # overrides existing output contents

  dataset="comics",                      # dataset name
  template="qwen",                     # use llama3 prompt template
  #train_on_prompt=True,
  val_size=0.1,
  max_samples=10000,                       # use 500 examples in each dataset

  finetuning_type="lora",                # use LoRA adapters to save memory
  lora_target="all",                     # attach LoRA adapters to all linear layers
  per_device_train_batch_size=2,         # the batch size
  gradient_accumulation_steps=4,         # the gradient accumulation steps
  lr_scheduler_type="cosine",            # use cosine learning rate scheduler
  loraplus_lr_ratio=16.0,                # use LoRA+ algorithm with lambda=16.0
  #temperature=0.5,
  
  warmup_ratio=0.1,                      # use warmup scheduler    
  learning_rate=5e-5,                    # the learning rate
  max_grad_norm=1.0,                     # clip gradient norm to 1.0
  
  fp16=True,                             # use float16 mixed precision training
  quantization_bit=4,                    # use 4-bit QLoRA  
  #use_liger_kernel=True,
  #quantization_device_map="auto",
  
  logging_steps=10,                      # log every 10 steps
  save_steps=5000,                       # save checkpoint every 1000 steps    
  logging_dir=str(LOGGING_DIR),
  
  # use_unsloth=True,
  report_to="tensorboard"                       # discards wandb

)

In [12]:
json.dump(args, open(train_file, "w", encoding="utf-8"), indent=2)

In [13]:
p = subprocess.Popen(["llamafactory-cli", "train", train_file], cwd=LLAMA_FACTORY_DIR)

In [14]:
p.wait()

11/08/2024 12:41:29 - INFO - llamafactory.cli - Initializing distributed tasks at: 127.0.0.1:22102


W1108 12:41:31.220000 139828114298176 torch/distributed/run.py:779] 
W1108 12:41:31.220000 139828114298176 torch/distributed/run.py:779] *****************************************
W1108 12:41:31.220000 139828114298176 torch/distributed/run.py:779] Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
W1108 12:41:31.220000 139828114298176 torch/distributed/run.py:779] *****************************************


11/08/2024 12:41:58 - WARNING - llamafactory.hparams.parser - We recommend enable `upcast_layernorm` in quantized training.
11/08/2024 12:41:58 - WARNING - llamafactory.hparams.parser - `ddp_find_unused_parameters` needs to be set as False for LoRA in DDP training.
11/08/2024 12:41:58 - INFO - llamafactory.hparams.parser - Process rank: 0, device: cuda:0, n_gpu: 1, distributed training: True, compute dtype: torch.float16


[INFO|configuration_utils.py:672] 2024-11-08 12:41:58,555 >> loading configuration file config.json from cache at /Utilisateurs/umushtaq/.cache/huggingface/hub/models--unsloth--Llama-3.2-1B-Instruct-bnb-4bit/snapshots/1f681675de2f6bcd23e4ec42837a48c84f55f960/config.json
[INFO|configuration_utils.py:739] 2024-11-08 12:41:58,555 >> Model config LlamaConfig {
  "_name_or_path": "unsloth/Llama-3.2-1B-Instruct-bnb-4bit",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "eos_token_id": [
    128001,
    128008,
    128009
  ],
  "head_dim": 64,
  "hidden_act": "silu",
  "hidden_size": 2048,
  "initializer_range": 0.02,
  "intermediate_size": 8192,
  "max_position_embeddings": 131072,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 16,
  "num_key_value_heads": 8,
  "pretraining_tp": 1,
  "quantization_config": {
    "_load_in_4bit": true,
    "_load_in_8bit": fa

11/08/2024 12:41:58 - WARNING - llamafactory.hparams.parser - We recommend enable `upcast_layernorm` in quantized training.
11/08/2024 12:41:58 - WARNING - llamafactory.hparams.parser - `ddp_find_unused_parameters` needs to be set as False for LoRA in DDP training.
11/08/2024 12:41:58 - INFO - llamafactory.hparams.parser - Process rank: 3, device: cuda:3, n_gpu: 1, distributed training: True, compute dtype: torch.float16
11/08/2024 12:41:58 - WARNING - llamafactory.hparams.parser - We recommend enable `upcast_layernorm` in quantized training.
11/08/2024 12:41:58 - WARNING - llamafactory.hparams.parser - `ddp_find_unused_parameters` needs to be set as False for LoRA in DDP training.
11/08/2024 12:41:58 - INFO - llamafactory.hparams.parser - Process rank: 2, device: cuda:2, n_gpu: 1, distributed training: True, compute dtype: torch.float16
11/08/2024 12:41:58 - WARNING - llamafactory.hparams.parser - We recommend enable `upcast_layernorm` in quantized training.
11/08/2024 12:41:58 - WARN

...

{'train_runtime': 512.1812, 'train_samples_per_second': 10.09, 'train_steps_per_second': 0.25, 'train_loss': 0.3481541071087122, 'epoch': 7.88}


[INFO|configuration_utils.py:672] 2024-11-08 12:50:55,424 >> loading configuration file config.json from cache at /Utilisateurs/umushtaq/.cache/huggingface/hub/models--unsloth--Llama-3.2-1B-Instruct-bnb-4bit/snapshots/1f681675de2f6bcd23e4ec42837a48c84f55f960/config.json
[INFO|configuration_utils.py:739] 2024-11-08 12:50:55,425 >> Model config LlamaConfig {
  "_name_or_path": "meta-llama/Llama-3.2-1B-Instruct",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "eos_token_id": [
    128001,
    128008,
    128009
  ],
  "head_dim": 64,
  "hidden_act": "silu",
  "hidden_size": 2048,
  "initializer_range": 0.02,
  "intermediate_size": 8192,
  "max_position_embeddings": 131072,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 16,
  "num_key_value_heads": 8,
  "pretraining_tp": 1,
  "quantization_config": {
    "_load_in_4bit": true,
    "_load_in_8bit": false,
 

***** train metrics *****
  epoch                    =     7.8769
  total_flos               = 15175537GF
  train_loss               =     0.3482
  train_runtime            = 0:08:32.18
  train_samples_per_second =      10.09
  train_steps_per_second   =       0.25


[INFO|modelcard.py:449] 2024-11-08 12:50:56,717 >> Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Causal Language Modeling', 'type': 'text-generation'}}


0

In [ ]:
args = dict(
  model_name_or_path=BASE_MODEL, # use bnb-4bit-quantized Llama-3-8B-Instruct model
  adapter_name_or_path=str(OUTPUT_DIR),            # load the saved LoRA adapters
  template="qwen",                     # same to the one in training
  finetuning_type="lora",                  # same to the one in training
  quantization_bit=4,                    # load 4-bit quantized model
)

In [16]:
model = ChatModel(args)

[INFO|configuration_utils.py:672] 2024-11-08 12:51:02,315 >> loading configuration file config.json from cache at /Utilisateurs/umushtaq/.cache/huggingface/hub/models--unsloth--Llama-3.2-1B-Instruct-bnb-4bit/snapshots/1f681675de2f6bcd23e4ec42837a48c84f55f960/config.json
[INFO|configuration_utils.py:739] 2024-11-08 12:51:02,317 >> Model config LlamaConfig {
  "_name_or_path": "unsloth/Llama-3.2-1B-Instruct-bnb-4bit",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "eos_token_id": [
    128001,
    128008,
    128009
  ],
  "head_dim": 64,
  "hidden_act": "silu",
  "hidden_size": 2048,
  "initializer_range": 0.02,
  "intermediate_size": 8192,
  "max_position_embeddings": 131072,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 16,
  "num_key_value_heads": 8,
  "pretraining_tp": 1,
  "quantization_config": {
    "_load_in_4bit": true,
    "_load_in_8bit": fa

11/08/2024 12:51:03 - INFO - llamafactory.data.template - Replace eos token: <|eot_id|>


[INFO|configuration_utils.py:672] 2024-11-08 12:51:03,883 >> loading configuration file config.json from cache at /Utilisateurs/umushtaq/.cache/huggingface/hub/models--unsloth--Llama-3.2-1B-Instruct-bnb-4bit/snapshots/1f681675de2f6bcd23e4ec42837a48c84f55f960/config.json
[INFO|configuration_utils.py:739] 2024-11-08 12:51:03,884 >> Model config LlamaConfig {
  "_name_or_path": "unsloth/Llama-3.2-1B-Instruct-bnb-4bit",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "eos_token_id": [
    128001,
    128008,
    128009
  ],
  "head_dim": 64,
  "hidden_act": "silu",
  "hidden_size": 2048,
  "initializer_range": 0.02,
  "intermediate_size": 8192,
  "max_position_embeddings": 131072,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 16,
  "num_key_value_heads": 8,
  "pretraining_tp": 1,
  "quantization_config": {
    "_load_in_4bit": true,
    "_load_in_8bit": fa

11/08/2024 12:51:03 - WARNING - llamafactory.model.model_utils.quantization - `quantization_bit` will not affect on the PTQ-quantized models.
11/08/2024 12:51:03 - INFO - llamafactory.model.model_utils.quantization - Loading ?-bit BITSANDBYTES-quantized model.
11/08/2024 12:51:03 - INFO - llamafactory.model.patcher - Using KV cache for faster generation.


[WARNING|quantization_config.py:400] 2024-11-08 12:51:03,997 >> Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
[INFO|modeling_utils.py:3726] 2024-11-08 12:51:04,210 >> loading weights file model.safetensors from cache at /Utilisateurs/umushtaq/.cache/huggingface/hub/models--unsloth--Llama-3.2-1B-Instruct-bnb-4bit/snapshots/1f681675de2f6bcd23e4ec42837a48c84f55f960/model.safetensors
[INFO|modeling_utils.py:1622] 2024-11-08 12:51:04,226 >> Instantiating LlamaForCausalLM model under default dtype torch.bfloat16.
[INFO|configuration_utils.py:1099] 2024-11-08 12:51:04,228 >> Generate config GenerationConfig {
  "bos_token_id": 128000,
  "eos_token_id": [
    128001,
    128008,
    128009
  ]
}

[INFO|quantizer_bnb_4bit.py:122] 2024-11-08 12:51:04,286 >> target_dtype {target_dtype} is replaced by `CustomDtype.INT4` for 4-bit BnB quantization
[INFO|modeling_utils.py:4568] 2024

11/08/2024 12:51:05 - INFO - llamafactory.model.model_utils.attention - Using torch SDPA for faster training and inference.
11/08/2024 12:51:05 - INFO - llamafactory.model.adapter - Loaded adapter(s): /Utilisateurs/umushtaq/emotion_analysis_comics/finetuning/saved_models/comics35_pg_nb_Llama-3.2-1B-Instruct-bnb-4bit
11/08/2024 12:51:05 - INFO - llamafactory.model.loader - all params: 1,241,450,496


In [17]:
model.engine

In [18]:
model.engine_type

'huggingface'

In [19]:
with open(test_dataset_file, "r+") as fh:
    test_dataset = json.load(fh)

test_prompts = []
test_grounds = []

for sample in test_dataset:
    test_prompts.append(sample["instruction"] + sample["input"])
    test_grounds.append(sample["output"])

In [20]:
test_predictions = []

In [21]:
messages = []

for prompt in tqdm(test_prompts, desc="Running inferences ..."):
    #print(type(prompt))
    #messages.append({"role": "user", "content": prompt})
    message = [{"role": "user", "content": prompt}]
    test_predictions.append(model.chat(message))

Running inferences ...:   0%|          | 0/156 [00:00<?, ?it/s]

[WARNING|logging.py:328] 2024-11-08 12:51:06,081 >> Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


Running inferences ...: 100%|██████████| 156/156 [05:04<00:00,  1.95s/it]


In [22]:
len(test_predictions)

156

In [23]:
test_predictions

[[Response(response_text='{"page_utterance_emotions": [["joy"], ["joy"], ["neutral"], ["neutral"], ["neutral"], ["joy"], ["fear", "surprise"], ["joy"], ["joy"], ["joy"], ["joy"]]}', response_length=48, prompt_length=381, finish_reason='stop')],
 [Response(response_text='{"page_utterance_emotions": [["joy"], ["fear", "surprise"], ["anger"], ["anger"], ["joy"], ["joy"], ["anger"]]}', response_length=36, prompt_length=525, finish_reason='stop')],
 [Response(response_text='{"page_utterance_emotions": [["sadness"], ["sadness"], ["sadness"], ["surprise"], ["neutral"], ["neutral"], ["anger", "surprise"], ["anger", "surprise"], ["joy"], ["surprise"], ["joy"], ["anger"], ["anger", "joy"], ["joy"], ["joy"], ["anger"]]}', response_length=74, prompt_length=620, finish_reason='stop')],
 [Response(response_text='{"page_utterance_emotions": [["anger", "sadness"], ["anger", "sadness"], ["anger", "sadness"], ["anger", "sadness"], ["anger", "sadness"], ["anger", "sadness"], ["anger", "sadness"], ["anger

In [24]:
processed_preds = []

for i, raw_pred in enumerate(test_predictions):
    try:
        x = json.loads(raw_pred[0].response_text)["page_utterance_emotions"]
        processed_preds.append(x)
    except:
        print(i)

In [25]:
len(processed_preds)

156

In [26]:
processed_grounds = []

for ground in test_grounds:
    x = json.loads(ground)["page_utterance_emotions"]
    processed_grounds.append(x)

In [27]:
len(processed_grounds)

156

In [28]:
bad_idx = []

for idx, (i,j) in enumerate(zip(processed_grounds, processed_preds)):
    if len(i) != len(j):
        print(idx, len(i), len(j))
        bad_idx.append(idx)

3 21 23
5 20 21
14 23 20
15 23 27
26 13 14
27 17 19
36 17 16
45 13 14
87 15 16
119 13 14
120 17 16
121 19 18
126 13 14
127 12 11


In [29]:
bad_idx.sort(reverse=True)

# Remove elements from 'grounds' at the specified indices
for idx in bad_idx:
    
    del processed_grounds[idx]
    del processed_preds[idx]

In [30]:
len(processed_grounds), len(processed_preds)

(142, 142)

In [31]:
grounds = [item for sublist in processed_grounds for item in sublist]
predictions = [item for sublist in processed_preds for item in sublist]

In [32]:
len(grounds), len(predictions)

(1090, 1090)

In [33]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()

In [34]:
y_true_mhot = mlb.fit_transform(grounds)
y_pred_mhot = mlb.transform(predictions)

In [35]:
y_true_mhot.shape, y_pred_mhot.shape

((1090, 7), (1090, 7))

In [36]:
class_labels = mlb.classes_

In [37]:
class_labels

array(['anger', 'disgust', 'fear', 'joy', 'neutral', 'sadness',
       'surprise'], dtype=object)

In [38]:
print(classification_report(y_true_mhot, y_pred_mhot, target_names=class_labels, digits=3))

              precision    recall  f1-score   support

       anger      0.489     0.538     0.512       398
     disgust      0.250     0.175     0.206        40
        fear      0.330     0.422     0.371       232
         joy      0.450     0.440     0.445       243
     neutral      0.241     0.169     0.198        77
     sadness      0.472     0.449     0.461       267
    surprise      0.490     0.507     0.498       294

   micro avg      0.439     0.456     0.448      1551
   macro avg      0.389     0.386     0.384      1551
weighted avg      0.438     0.456     0.445      1551
 samples avg      0.457     0.468     0.437      1551



In [39]:
# with open({Path(FT_DIR)} / "classification_report.pickle", 'wb') as fh:
    
#      pickle.dump(classification_report(y_true_mhot, y_pred_mhot, target_names=class_labels, digits=3, output_dict=True), fh)